In [2]:
import math
import numpy as np
from numpy import  linalg as LA


class MaxEntropy:
    def __init__(self, dataset, epsilon=1e-3, maxstep=4, rate=0.1):
        self._samples = dataset # 训练数据集
        self.epsilon = epsilon # 停止迭代条件
        self.maxstep = maxstep # 最大训练次数
        self._Y = set()  # 训练数据包含的类
        self._numXY = {}  # key:特征函数(x/特征, y/列),value:该特征函数在训练数据集中出现的次数
        self._N = 0  # 训练数据样本个数
        self._n = 0 # 特征函数个数
        self._C = 0 # 最大特征数
        self._w = None # 权值向量 
        self._Ep_ = [] # 特征函数的概率分布(也等于特征函数关于经验分布\hat{P}f(X, Y))的期望值
        self._xyID = {} # key:特征函数,value:id
        self._IDxy = {} # key:id,value:特征函数
        self._B0 = None # BFGS算法中的B0矩阵
        self.rate = rate # BFGS算法中一维搜索的步长

         
    def init_param(self):
        """模型参数初始化"""
        for items in self._samples:
            y = items[0] # 实例的类别
            X = items[1:] # 实例特征向量
            self._Y.add(y) # 添加类y,若类y已存在则忽略
            for x in X:
                if (x, y) in self._numXY:
                    self._numXY[(x, y)] += 1
                else:
                    self._numXY[(x, y)] = 1

        self._N = len(self._samples)
        self._n = len(self._numXY)
        self._B0 = np.eye(self._n) # 初始化的单位矩阵(满足正定对称矩阵的要求)
        self._C = max([len(sample) - 1 for sample in self._samples])
        self._w = np.zeros(self._n) # 权值向量初始为0向量
        self._Ep_ = [0] * self._n # 初始化为0向量
        for i, xy in enumerate(self._numXY):
            self._Ep_[i] = self._numXY[xy] / self._N
            self._xyID[xy] = i
            self._IDxy[i] = xy
    
    
    def _Zx(self, 
            X): # X为每一条训练数据集
        """计算Z_w(x),Z_w(x)为一个值"""
        zx = 0
        for y in self._Y:
            ss = 0
            for x in X:
                if (x, y) in self._numXY: # 如果(\mathbf{x}, y)是特征函数
                    ss += self._w[self._xyID[(x, y)]]
            zx += math.exp(ss)
            
        return zx
 
 
    def _model_pyx(self, 
                   y, # y的可能取值
                   X): # X为每一条训练数据集
        """计算p_w(y|x),注意:\sum_{y} p_w(y|x)=1,p_w(y|x)为概率分布"""
        zx = self._Zx(X)
        ss = 0
        for x in X:
            if (x, y) in self._numXY:
                ss += self._w[self._xyID[(x, y)]]
        pyx = math.exp(ss) / zx
        
        return pyx
    
    
    def _model_ep(self, index):
        """计算某个特征函数关于模型的期望值"""
        x, y = self._IDxy[index] # 第i个特征函数
        ep = 0
        for sample in self._samples:
            if x not in sample:
                continue
            pyx = self._model_pyx(y, sample)
            ep += pyx / self._N
            
        return ep

    
    def func_grad(self):
        """计算目标函数关于权值向量的梯度"""
        pxy_lst = list()
        for i in range(self._n):
            pxy_lst.append(self._model_ep(i))

        return  np.array(pxy_lst) - self._Ep_ # 梯度
   
   
    def updataB0(self, delta_lst, y_lst):
        """更新正定矩阵B0"""
        delta = (delta_lst[1] - delta_lst[0]).reshape(-1, 1)
        y = (y_lst[1] - y_lst[0]).reshape(-1, 1)
        part1 = (y @ y.T) / (y.T @ delta)
        part2 = (self._B0 @ delta @ delta.T @ self._B0) / (delta.T @ self._B0 @ delta)
        self._B0 = self._B0 + part1 - part2
    
    
    def BFGS(self):
        """最大熵模型学习的BFGS算法"""
        self.init_param() # 模型初始化
        g_k0 = self.func_grad()
        y_vec = g_k0
        distance0 = LA.norm(g_k0, ord=2) # L2范数,用来表示向量的大小
        w_lst = [self._w, self._w] # 保留此次和上次sefl._w(权值向量)
        g_k_lst = [g_k0, g_k0] # 保留此次和上次的g_k(梯度)
        if distance0 < self.epsilon:
            return self._w
        else:
            for i in range(self.maxstep):
                p_k = -LA.solve(self._B0, y_vec)
                self._w = self._w + self.rate*p_k 
                w_lst.pop(0)
                w_lst.insert(1, self._w)
                w_lst[1] = self._w
                g_k1 = self.func_grad()
                y_vec = g_k1
                g_k_lst.pop(0)
                g_k_lst.insert(1, g_k1)
                distance1 = LA.norm(g_k1, ord=2)
                if distance1 < self.epsilon:
                    return self._w
                else:
                    self.updataB0(w_lst, g_k_lst) # 更新B0
                    
                    
    def predict(self, X):
        """计算测试数据集的概率"""
        Z = self._Zx(X)
        result = {}
        for y in self._Y:
            ss = 0
            for x in X:
                if (x, y) in self._numXY:
                    ss += self._w[self._xyID[(x, y)]]
            pyx = math.exp(ss) / Z
            result[y] = pyx # 通过p_w(y|\mathcal{x})计算得(权值向量\mathbf{w}是最大熵模型中的参数向量)
        return result


In [3]:
# 训练数据集
data = [['no', 'sunny', 'hot', 'high', 'FALSE'],
           ['no', 'sunny', 'hot', 'high', 'TRUE'],
           ['yes', 'overcast', 'hot', 'high', 'FALSE'],
           ['yes', 'rainy', 'mild', 'high', 'FALSE'],
           ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
           ['no', 'rainy', 'cool', 'normal', 'TRUE'],
           ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
           ['no', 'sunny', 'mild', 'high', 'FALSE'],
           ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
           ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
           ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
           ['yes', 'overcast', 'mild', 'high', 'TRUE'],
           ['yes', 'overcast', 'hot', 'normal', 'FALSE'],
           ['no', 'rainy', 'mild', 'high', 'TRUE']]

In [4]:
maxent = MaxEntropy(dataset=data, epsilon=1e-2, maxstep=100)
p_x = ['overcast', 'mild', 'high', 'FALSE'] # 测试数据集

In [5]:
last_w = maxent.BFGS()
print('最终的权值向量为:\n', last_w)

最终的权值向量为:
 [ 5.71321912  0.96202027  3.7027961  -3.77182992  2.16855512 10.56620575
 -0.96202027 -3.7027961   3.77182992 -3.24971183  4.42819421 -1.86289914
  5.3060709   3.24971183  1.86289914 -5.3060709  -2.16855512 -4.42819421
 -5.71321912]


In [6]:
print('predict result:', maxent.predict(p_x)) # 极高的概率被认为是"yes"

predict result: {'yes': 0.9999999968017492, 'no': 3.1982508065415424e-09}


In [7]:
# 使用KNN算法与决策树算法进行验证
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import  DecisionTreeClassifier

In [8]:
arr_data = np.array(data)
enc = OrdinalEncoder() # 离散型数据编码
enc.fit(arr_data[:, 1:])
arr_data_int = enc.transform(arr_data[:, 1:])
arr_data_int

array([[2., 1., 0., 0.],
       [2., 1., 0., 1.],
       [0., 1., 0., 0.],
       [1., 2., 0., 0.],
       [1., 0., 1., 0.],
       [1., 0., 1., 1.],
       [0., 0., 1., 1.],
       [2., 2., 0., 0.],
       [2., 0., 1., 0.],
       [1., 2., 1., 0.],
       [2., 2., 1., 1.],
       [0., 2., 0., 1.],
       [0., 1., 1., 0.],
       [1., 2., 0., 1.]])

In [9]:
p_x_int = enc.transform(np.array(p_x).reshape(1, -1))
p_x_int

array([[0., 2., 0., 0.]])

In [10]:
kNN_classifier = KNeighborsClassifier(n_neighbors=2, # 临近点个数,即k值(默认n_neighbors=5) 
                                      weights='distance',
                                      p=2, # 选择何种Minkowski距离(默认p=2,即欧氏距离)
                                      n_jobs=-1)

In [11]:
kNN_classifier.fit(arr_data_int, arr_data[:, 0])
kNN_classifier.predict(p_x_int) # KNN算法预测结果为"yes"

array(['yes'], dtype='<U8')

In [12]:
clf = DecisionTreeClassifier()
clf.fit(arr_data_int, arr_data[:, 0])
clf.predict(p_x_int) # 决策树算法预测结果为"yes"

array(['yes'], dtype='<U8')